In [689]:
import numpy as np, pandas as pd, polars as pl
from sklearn.model_selection import KFold
from sklearn import preprocessing as skp
from sklearn.decomposition import PCA 

import xgboost as xgb
from sklearn.metrics import root_mean_squared_error as rmse

In [690]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

In [691]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 750000 non-null  object 
 1   Episode_Title                750000 non-null  object 
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object 
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object 
 6   Publication_Time             750000 non-null  object 
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object 
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 68.7+ MB


In [692]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250000 entries, 750000 to 999999
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 250000 non-null  object 
 1   Episode_Title                250000 non-null  object 
 2   Episode_Length_minutes       221264 non-null  float64
 3   Genre                        250000 non-null  object 
 4   Host_Popularity_percentage   250000 non-null  float64
 5   Publication_Day              250000 non-null  object 
 6   Publication_Time             250000 non-null  object 
 7   Guest_Popularity_percentage  201168 non-null  float64
 8   Number_of_Ads                250000 non-null  float64
 9   Episode_Sentiment            250000 non-null  object 
dtypes: float64(4), object(6)
memory usage: 21.0+ MB


In [693]:
categorical_map = {
        'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6, # Day

        'Morning': 0, 'Afternoon': 1, 'Evening': 2, 'Night': 3,                                            # Time

        'Negative': 0, 'Neutral': 1, 'Positive': 2,                                                        # Sentiment

        'True Crime': 0, 'Comedy': 1, 'Education': 2, 'Technology': 3, 'Health': 4, 'News': 5, 'Music': 6, # Genre
        'Sports': 7, 'Business': 8, 'Lifestyle': 9
        }

categories = ['Genre', 'Episode_Title', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

In [694]:
train.drop('Podcast_Name', inplace=True, axis=1)
test.drop('Podcast_Name', inplace=True, axis=1)

In [695]:
train['Episode_Title'] = train['Episode_Title'].str[8:]
test['Episode_Title'] = test['Episode_Title'].str[8:]

In [696]:
train = train.replace(categorical_map).fillna(0)
test = test.replace(categorical_map).fillna(0)

In [697]:
x = train.drop('Listening_Time_minutes', axis=1)
y = train['Listening_Time_minutes']

In [698]:
poly = skp.PolynomialFeatures(interaction_only=True, include_bias=False)
x_poly = poly.fit_transform(x)
test_poly = poly.fit_transform(test)

In [699]:
cols = poly.get_feature_names_out(x.columns)
x = pd.DataFrame(x_poly, columns=cols)

test_cols = poly.get_feature_names_out(test.columns)
test = pd.DataFrame(test_poly, columns=test_cols)

In [700]:
x = x.iloc[:,:23]
test = test.iloc[:,:23]

In [701]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 23 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   Episode_Title                                       750000 non-null  float64
 1   Episode_Length_minutes                              750000 non-null  float64
 2   Genre                                               750000 non-null  float64
 3   Host_Popularity_percentage                          750000 non-null  float64
 4   Publication_Day                                     750000 non-null  float64
 5   Publication_Time                                    750000 non-null  float64
 6   Guest_Popularity_percentage                         750000 non-null  float64
 7   Number_of_Ads                                       750000 non-null  float64
 8   Episode_Sentiment                                   750000 non-n

In [702]:
mm_scaler = skp.MinMaxScaler()
x_s = pd.DataFrame(mm_scaler.fit_transform(x), columns=x.columns, index=x.index)

In [703]:
for c in categories: 
    x_s[c] = x_s[c].astype('category')
    test[c] = test[c].astype('category')

In [704]:
K = 5
kf = KFold(K, shuffle=True, random_state=55)

In [705]:
pred = pd.DataFrame()
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [706]:
for train_idx, val_idx in kf.split(x,y):
    x_t, y_t = x_s.iloc[train_idx], y.iloc[train_idx]
    x_v, y_v = x_s.iloc[val_idx], y.iloc[val_idx]
    model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric = 'rmse', enable_categorical = True, device = 'cuda', tree_method="hist")#, min_child_weight=4)
    model.fit(x_t, y_t, eval_set=[(x_v,y_v)])
    pred = pd.concat([pd.DataFrame(model.predict(test)), pred], axis=1)
    #xgb.plot_importance(model.get_booster().get_score(importance_type='weight'), show_values=True)

[0]	validation_0-rmse:21.24415
[1]	validation_0-rmse:17.63748
[2]	validation_0-rmse:15.55008
[3]	validation_0-rmse:14.40762
[4]	validation_0-rmse:13.79975
[5]	validation_0-rmse:13.48350
[6]	validation_0-rmse:13.31783
[7]	validation_0-rmse:13.23089
[8]	validation_0-rmse:13.18172
[9]	validation_0-rmse:13.15380
[10]	validation_0-rmse:13.13824
[11]	validation_0-rmse:13.12607
[12]	validation_0-rmse:13.11718
[13]	validation_0-rmse:13.10960
[14]	validation_0-rmse:13.10403
[15]	validation_0-rmse:13.09948
[16]	validation_0-rmse:13.09816
[17]	validation_0-rmse:13.09604
[18]	validation_0-rmse:13.09375
[19]	validation_0-rmse:13.09099
[20]	validation_0-rmse:13.08834
[21]	validation_0-rmse:13.08529
[22]	validation_0-rmse:13.08486
[23]	validation_0-rmse:13.08408
[24]	validation_0-rmse:13.08328
[25]	validation_0-rmse:13.08381
[26]	validation_0-rmse:13.07907
[27]	validation_0-rmse:13.07728
[28]	validation_0-rmse:13.07597
[29]	validation_0-rmse:13.07318
[30]	validation_0-rmse:13.07121
[31]	validation_0-

In [707]:
pred.reset_index(drop=True, inplace=True)
pred.index += 750000
pred_sum = pred.sum(axis=1) / K

In [708]:
pred.head()

,0,0,0,0,0
750000,114.203751,89.461624,113.581955,75.274490,96.830185
750001,104.960159,107.082520,100.511871,96.679604,94.899773
750002,114.664154,106.824875,108.285332,94.125023,105.778839
750003,110.548653,101.127670,108.225998,88.166252,100.502670
750004,111.553833,91.977180,101.173286,107.149658,116.516418


In [709]:
pred_sum.rename('Listening_Time_minutes', inplace=True)
pred_sum.head()

750000     97.870399
750001    100.826790
750002    105.935646
750003    101.714256
750004    105.674072
Name: Listening_Time_minutes, dtype: float32

In [710]:
pred_sum.to_csv('submission.csv', index_label='id')